In [1]:
import pandas as pd
import numpy as np
import importlib
import awswrangler as wr
from application.modeling import helper
import os
from application.modeling.constants import MY_SESSION, S3_CLIENT, FileStage
from decouple import config
from application.utils import schemas
from application.routes.projects import crud
from application.utils.database import SessionLocal
from application.utils import models


pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:,.2f}".format

In [2]:
bucket_name = "claxon-actuaries-budgeting"
tenant_name = bucket_name
project_id = 21
# start_date = "2023-01"
# months_to_forecast = 12

In [3]:
[i for i in dir( wr.s3) if not i.startswith('_') ]

['copy_objects',
 'delete_objects',
 'describe_objects',
 'does_object_exist',
 'download',
 'get_bucket_region',
 'list_buckets',
 'list_directories',
 'list_objects',
 'merge_datasets',
 'merge_upsert_table',
 'read_csv',
 'read_excel',
 'read_fwf',
 'read_json',
 'read_parquet',
 'read_parquet_metadata',
 'read_parquet_table',
 'select_query',
 'size_objects',
 'store_parquet_metadata',
 'to_csv',
 'to_excel',
 'to_json',
 'to_parquet',
 'upload',
 'wait_objects_exist',
 'wait_objects_not_exist']

In [6]:
file_stage = FileStage.raw
expenses_uncertain = wr.s3.read_parquet(
    f"s3://{bucket_name}/project_{project_id}/{file_stage.value}/expenses_uncertain.parquet",
    boto3_session=MY_SESSION,
)

expenses_uncertain

,Unnamed:_0,1,2,3,4,5,6
0,Travel Costs,0.00,0.00,0.00,0.00,0.00,0.00
1,Entertainment,0.00,721.39,67.00,0.00,"4,530.00",124.00
2,Marketing Costs,"3,104.93","4,869.97","8,450.63","4,098.90","5,692.00",195.60
3,Group Marketing Costs,"7,145.92","7,543.70","12,847.68","9,277.20","6,097.62","15,314.38"
4,Rental Costs,910.55,"1,168.40",583.42,"1,479.51","1,601.02","1,315.55"
5,Subscriptions,295.54,272.00,463.35,"1,414.76",270.26,725.50
6,Insurance,"1,593.96","1,386.05","3,013.06","1,151.40","1,151.40","6,168.06"
7,Repairs And Maintenance,801.30,97.20,0.00,725.79,"2,222.33","2,721.10"
8,Utilities,"1,575.33","2,325.66","1,801.68",993.94,"1,142.33",468.39
9,Stationery,"1,843.48",246.10,18.18,"2,344.41",199.32,185.13
